In [1]:
import time
import threading
from threading import Thread
import random
import sys

# Shared Data and Synchronization

As we saw in our previous notebook, threads can access to share data. This is useful to communicate things and access common variables. But it'll also introduce problems, mainly [**Race Conditions**](https://en.wikipedia.org/wiki/Race_condition). Let's see an example of a Race Condition.

In [2]:
COUNTER = 0

In [3]:
def increment(n):
    global COUNTER
    for _ in range(n):
        COUNTER += 1
        print(COUNTER)
        time.sleep(0.001)

In [4]:
ITERATIONS = 1000

In [5]:
threads = [Thread(target=increment, args=(ITERATIONS,)) for _ in range(10)]

In [6]:
[t.start() for t in threads];

1
2
3
4
56

78
9
10
11

12
13
14
15
16
1718
1920
212223


24

25
26
272829


30

3132333435363738
39





40414243
4445


46
474849


50




51
5253545556575859







6061


In [7]:
assert COUNTER == (len(threads) * ITERATIONS), f"Invalid value for counter: {COUNTER}"


6263646566676869







70
71
72737475767778






7980

81
82838485868788899091









9293949596979899100101









102103104105106107108109

110






111
112
113114115116117118
119





120121

122
123124125126127128


129



130131

132133134135136137138139







140141

142143144145146147148149







150151

152153154155156157158159







160161

162163164165166167168169







170171

172173174175176177178179







180181

182183184185186187188189







190191

192193194195196197198199







200201

202203
204205206207

208


209

210211

212213
214215
216217
218

219


220221

222
223224225226227228


229



230231

232233234235236237238239







240241

242243244245246247248249







250251

252253254255256257258259




260261




262263264


265266267268269270271






272273274


275276277278279280281






282283284


285286287288289290291






292293294


295
296297298299300

301


302303304



305306307308309




310311312313314




315316317318319




32032

AssertionError: Invalid value for counter: 78


678

679680681682
683
684



685686687688



689690691692693694





695696697698



699700701702703704





705706707708


709710711712713
714





715716717

718719720721722723724







725726



What's happening here is that the operation `+=` is not **atomic**. Which means that, behind the scenes, `+=` runs 2 (or more) operations:

```python
aux = COUNTER + 1
COUNTER = aux
```

Both threads are executing concurrently (potentially at the same time) and they're reading outdated values of `COUNTER`, which results in a race condition.

## Thread Synchronization

How can we fix then this race condition? Basically, we need a way to keep the threads from stepping onto each other's data, some signal that the given resource _"is busy"_:

<img src="img/recording_studio_light.png" width=400/>

*(Example of INEs studios, a recording light is on, the studio is busy, nobody will enter the room)*

The easiest synchronization mechanism is a [_Lock_](https://en.wikipedia.org/wiki/Lock_(computer_science)), or a _Mutex_ (mutual exclusion lock). Python includes the very intuitive `threading.Lock` class. Let's see how a Lock works.

A Lock works in the same way as the Studio Light from the picture. The first one that "arrives" to that given resource "turns on the light", or, formally, "acquires the lock". Any other threads reaching that point, if they want to acquire the lock, they have to wait for the first thread to "release it". Let's see an example:

## Communicaion Between Thread
The safest way to send data from one thread to another is to use __Queue__ object from queue libarary. To do this, you create Queue objectthat is shared by the threads. Threads then use put() or get() operations to add or remove items from the queue.

In [20]:
from queue import Queue
def producer(out_queue):
    print("Thread {} produces new_value".format(threading.current_thread().name))
    out_queue.put(5)

def consumer(in_queue):
    data=in_queue.get()
    print("Tread {} consumes data {}".format(threading.current_thread().name,data))

starting_queue=Queue()
t1=Thread(target=producer,args=(starting_queue,))
t2=Thread(target=consumer,args=(starting_queue,))
t1.start()
t2.start()    

Thread Thread-23 produces new_value
Tread Thread-24 consumes data 5


#### Locking

In [ ]:
lock = threading.Lock()

In [ ]:
def lock_hogger(lock, sleep=10):
    print("\t\tThread: Acquiring lock.")
    lock.acquire()
    print("\t\tThread: Lock acquired, sleeping")
    if sleep:
        time.sleep(sleep)
    print("\t\tThread: Woke up, releasing lock")
    lock.release()

In [ ]:
t = Thread(target=lock_hogger, args=(lock, ))

In [ ]:
t.start()

Trying to acquire the lock here will probably block for a few seconds:

In [ ]:
lock.locked()

In [ ]:
lock.acquire()
print("Lock acquired!")

Once the lock has been acquired, any other thread that tries to acquire it will block:

In [ ]:
t = Thread(target=lock_hogger, args=(lock, 0))

In [ ]:
t.start()

In [ ]:
lock.release()

#### Time to fix our counter!

Now that we know about locks, we can use them to fix our counter example:

In [ ]:
COUNTER = 0

In [ ]:
def increment(n, lock):
    global COUNTER
    for _ in range(n):
        lock.acquire()
        COUNTER += 1
        print("Thread Name:{}-> Counter:{}".format(threading.current_thread().name,COUNTER))
        lock.release()
        time.sleep(0.001)

In [ ]:
ITERATIONS = 1000

In [ ]:
lock = threading.Lock()

727728729730731732733734







735736

737738739740741742743744







745
746
747748749750751


752
753


754
755
756757758759760761762






763
764765

766767768769770771772
773774775








776777778779780781782783
784





785


786787788789790791792793794




795




796797798799800801802803804805









806807808809810811812813

814



815



816817818819820821822823824825









826827
828829830831832833834835








836837838839
840841842843

844


845



846847848849850
851852853854855









In [ ]:
threads = [Thread(target=increment, args=(ITERATIONS, lock)) for _ in range(10)]

In [ ]:
[t.start() for t in threads];
[t.join() for t in threads];

In [ ]:
COUNTER

In [ ]:
assert COUNTER == (len(threads) * ITERATIONS), f"Invalid value for counter: {COUNTER}"

It doesn't matter how many times we run the example, our code will always be synchronized!

#### Problems with synchronization

Locks are acquired before accessing what we call "Critical Sections"; important sections in our code that can potentially introduce race conditions. The usual process is:

```python
lock = threading.Lock() # Problem [1]

# before entering critical section
lock.acquire()          # Problem [2]

# critical section
do_your_thing()         # Problem [3]

# after we're done with it
lock.release()          # Problem [4]
```

The problem is that locks (and many other synchronization mechanisms) are "cooperative". You're cooperating by using locks, but you're not obliged to use them. In a team of n developers, just one of them screws up with their lock management, everybody loses.

These are the things that can potentially go wrong with cooperative, manual synchronization mechanisms:

1. You might forget to use locks at all! You might have failed to recognize the situation as having a "critical section".
2. You might forget to acquire the lock, getting directly into the critical section.
3. Your critical section might be using resources NOT protected by the lock you're using, so other threads can be stepping onto that.
4. You might forget to release the lock, or your code could break before you're able to release the lock (next example)
5. Deadlocks! (more on later).

Problem *No.4* is very common, let's see an example of it:

In [10]:
lock = threading.Lock()

In [11]:
def faulty_lock_handler(lock, sleep=10):
    print("\t\tThread: Acquiring lock.")
    lock.acquire()
    print("\t\tThread: Lock acquired")
    if sleep:
        time.sleep(sleep)
    print("\t\tThread: Woke up, releasing lock")
    lock.release()

In [12]:
t = Thread(target=faulty_lock_handler, args=(lock, 'x'))  # The `sleep` param is incorrect, should be a number

In [ ]:
t.start()


856857858859860861862863864








865
866867868869870871872873




874


875

876877878879



880881882883884




885
886887888
889890
891892893894


895




896
897898899900901902903904905







906

907908909910911912913914




915



916917918919920921922923924







925



Trying to acquire the lock will block **FOREVER**:

In [15]:
lock.acquire?

We could potentially add a _timeout_ to our acquire method, it will block for `n` seconds and if it hasn't acquired the lock, it'll return `False`:

In [14]:
lock.acquire(timeout=2)

False

Or we can even make it non-blocking, if it's not able to acquire the lock, it'll release it immediately:

In [ ]:
lock.acquire(blocking=False)

Thankfully, we have a handle on the lock variable, so we can release it from here, but this is cheating:

In [ ]:
lock.release()

In [ ]:
lock.acquire(blocking=False)

The way to solve this is to use Locks as Context Managers, so we're sure we'll release the lock EVEN if something goes wrong within the critical section:

In [4]:
lock = threading.Lock()

In [5]:
def fixed_lock_handler(lock, sleep=10):
    print("\t\tThread: Acquiring lock.")
    with lock:
        print("\t\tThread: Lock acquired")
        if sleep:
            time.sleep(sleep)
    print("\t\tThread: Woke up, releasing lock")

In [6]:
t = Thread(target=fixed_lock_handler, args=(lock, 5))

In [7]:
t.start()

		Thread: Acquiring lock.
		Thread: Lock acquired


In [8]:
lock.acquire()

		Thread: Woke up, releasing lock


True

In [9]:
lock.release()

In [10]:
t = Thread(target=fixed_lock_handler, args=(lock, 'x'))  # The `sleep` param is incorrect, should be a number

In [11]:
t.start()

		Thread: Acquiring lock.
		Thread: Lock acquired


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\pc\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\pc\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-9d5dd1ab9e64>", line 6, in fixed_lock_handler
    time.sleep(sleep)
TypeError: an integer is required (got type str)



Is the lock still acquired?

In [12]:
lock.locked()

False

In [ ]:
lock.acquire()

In [ ]:
lock.release()

As you can see, the critical section failed with an exception, but the lock was released before exiting. As a reference, the `with` context manager is syntactic sugar for the pattern:

```python
lock.acquire()
try:
    critical_section()
finally:
    lock.release()  # We'll release the lock no matter what
```

#### Fixing our counter using `with`

The last touch for our counter should be to use the context manager protocol of the lock object:

In [25]:
COUNTER = 0

In [26]:
def increment(n, lock):
    global COUNTER
    for _ in range(n):
        with lock:
            COUNTER += 1
            print("Thread Name:{}-> Counter:{}".format(threading.current_thread().name,COUNTER))            
            time.sleep(0.001)

In [27]:
ITERATIONS = 1000

In [28]:
lock = threading.Lock()

In [29]:
threads = [Thread(target=increment, args=(ITERATIONS, lock)) for _ in range(10)]

In [30]:
[t.start() for t in threads];

Thread Name:Thread-28-> Counter:1
Thread Name:Thread-28-> Counter:2
Thread Name:Thread-28-> Counter:3
Thread Name:Thread-28-> Counter:4
Thread Name:Thread-28-> Counter:5
Thread Name:Thread-33-> Counter:6
Thread Name:Thread-34-> Counter:7
Thread Name:Thread-34-> Counter:8
Thread Name:Thread-34-> Counter:9
Thread Name:Thread-34-> Counter:10
Thread Name:Thread-37-> Counter:11
Thread Name:Thread-30-> Counter:12
Thread Name:Thread-31-> Counter:13
Thread Name:Thread-32-> Counter:14
Thread Name:Thread-28-> Counter:15
Thread Name:Thread-33-> Counter:16
Thread Name:Thread-29-> Counter:17
Thread Name:Thread-35-> Counter:18
Thread Name:Thread-35-> Counter:19
Thread Name:Thread-35-> Counter:20
Thread Name:Thread-35-> Counter:21
Thread Name:Thread-35-> Counter:22
Thread Name:Thread-35-> Counter:23
Thread Name:Thread-35-> Counter:24
Thread Name:Thread-35-> Counter:25
Thread Name:Thread-35-> Counter:26
Thread Name:Thread-35-> Counter:27
Thread Name:Thread-35-> Counter:28
Thread Name:Thread-35-> Count

Thread Name:Thread-33-> Counter:257
Thread Name:Thread-33-> Counter:258
Thread Name:Thread-33-> Counter:259
Thread Name:Thread-33-> Counter:260
Thread Name:Thread-33-> Counter:261
Thread Name:Thread-33-> Counter:262
Thread Name:Thread-33-> Counter:263
Thread Name:Thread-33-> Counter:264
Thread Name:Thread-33-> Counter:265
Thread Name:Thread-33-> Counter:266
Thread Name:Thread-33-> Counter:267
Thread Name:Thread-33-> Counter:268
Thread Name:Thread-33-> Counter:269
Thread Name:Thread-33-> Counter:270
Thread Name:Thread-33-> Counter:271
Thread Name:Thread-35-> Counter:272
Thread Name:Thread-35-> Counter:273
Thread Name:Thread-35-> Counter:274
Thread Name:Thread-35-> Counter:275
Thread Name:Thread-35-> Counter:276
Thread Name:Thread-35-> Counter:277
Thread Name:Thread-35-> Counter:278
Thread Name:Thread-35-> Counter:279
Thread Name:Thread-35-> Counter:280
Thread Name:Thread-35-> Counter:281
Thread Name:Thread-35-> Counter:282
Thread Name:Thread-35-> Counter:283
Thread Name:Thread-35-> Coun

Thread Name:Thread-30-> Counter:537
Thread Name:Thread-30-> Counter:538
Thread Name:Thread-30-> Counter:539
Thread Name:Thread-30-> Counter:540
Thread Name:Thread-30-> Counter:541
Thread Name:Thread-30-> Counter:542
Thread Name:Thread-30-> Counter:543
Thread Name:Thread-30-> Counter:544
Thread Name:Thread-31-> Counter:545
Thread Name:Thread-31-> Counter:546
Thread Name:Thread-31-> Counter:547
Thread Name:Thread-34-> Counter:548
Thread Name:Thread-29-> Counter:549
Thread Name:Thread-29-> Counter:550
Thread Name:Thread-29-> Counter:551
Thread Name:Thread-29-> Counter:552
Thread Name:Thread-29-> Counter:553
Thread Name:Thread-29-> Counter:554
Thread Name:Thread-29-> Counter:555
Thread Name:Thread-29-> Counter:556
Thread Name:Thread-29-> Counter:557
Thread Name:Thread-29-> Counter:558
Thread Name:Thread-29-> Counter:559
Thread Name:Thread-29-> Counter:560
Thread Name:Thread-29-> Counter:561
Thread Name:Thread-29-> Counter:562
Thread Name:Thread-29-> Counter:563
Thread Name:Thread-29-> Coun

Thread Name:Thread-37-> Counter:823
Thread Name:Thread-28-> Counter:824
Thread Name:Thread-28-> Counter:825
Thread Name:Thread-28-> Counter:826
Thread Name:Thread-28-> Counter:827
Thread Name:Thread-28-> Counter:828
Thread Name:Thread-28-> Counter:829
Thread Name:Thread-28-> Counter:830
Thread Name:Thread-28-> Counter:831
Thread Name:Thread-28-> Counter:832
Thread Name:Thread-28-> Counter:833
Thread Name:Thread-28-> Counter:834
Thread Name:Thread-28-> Counter:835
Thread Name:Thread-31-> Counter:836
Thread Name:Thread-31-> Counter:837
Thread Name:Thread-34-> Counter:838
Thread Name:Thread-33-> Counter:839
Thread Name:Thread-30-> Counter:840
Thread Name:Thread-30-> Counter:841
Thread Name:Thread-30-> Counter:842
Thread Name:Thread-30-> Counter:843
Thread Name:Thread-30-> Counter:844
Thread Name:Thread-30-> Counter:845
Thread Name:Thread-30-> Counter:846
Thread Name:Thread-30-> Counter:847
Thread Name:Thread-30-> Counter:848
Thread Name:Thread-30-> Counter:849
Thread Name:Thread-30-> Coun

Thread Name:Thread-28-> Counter:1105
Thread Name:Thread-28-> Counter:1106
Thread Name:Thread-28-> Counter:1107
Thread Name:Thread-28-> Counter:1108
Thread Name:Thread-28-> Counter:1109
Thread Name:Thread-28-> Counter:1110
Thread Name:Thread-28-> Counter:1111
Thread Name:Thread-37-> Counter:1112
Thread Name:Thread-37-> Counter:1113
Thread Name:Thread-36-> Counter:1114
Thread Name:Thread-36-> Counter:1115
Thread Name:Thread-36-> Counter:1116
Thread Name:Thread-29-> Counter:1117
Thread Name:Thread-29-> Counter:1118
Thread Name:Thread-32-> Counter:1119
Thread Name:Thread-32-> Counter:1120
Thread Name:Thread-32-> Counter:1121
Thread Name:Thread-32-> Counter:1122
Thread Name:Thread-32-> Counter:1123
Thread Name:Thread-32-> Counter:1124
Thread Name:Thread-32-> Counter:1125
Thread Name:Thread-32-> Counter:1126
Thread Name:Thread-32-> Counter:1127
Thread Name:Thread-32-> Counter:1128
Thread Name:Thread-32-> Counter:1129
Thread Name:Thread-32-> Counter:1130
Thread Name:Thread-32-> Counter:1131
T

...

Thread Name:Thread-36-> Counter:8671
Thread Name:Thread-36-> Counter:8672
Thread Name:Thread-36-> Counter:8673
Thread Name:Thread-36-> Counter:8674
Thread Name:Thread-36-> Counter:8675
Thread Name:Thread-33-> Counter:8676
Thread Name:Thread-33-> Counter:8677
Thread Name:Thread-33-> Counter:8678
Thread Name:Thread-33-> Counter:8679
Thread Name:Thread-33-> Counter:8680
Thread Name:Thread-31-> Counter:8681
Thread Name:Thread-36-> Counter:8682
Thread Name:Thread-36-> Counter:8683
Thread Name:Thread-36-> Counter:8684
Thread Name:Thread-36-> Counter:8685
Thread Name:Thread-36-> Counter:8686
Thread Name:Thread-36-> Counter:8687
Thread Name:Thread-36-> Counter:8688
Thread Name:Thread-36-> Counter:8689
Thread Name:Thread-34-> Counter:8690
Thread Name:Thread-34-> Counter:8691
Thread Name:Thread-34-> Counter:8692
Thread Name:Thread-34-> Counter:8693
Thread Name:Thread-34-> Counter:8694
Thread Name:Thread-34-> Counter:8695
Thread Name:Thread-34-> Counter:8696
Thread Name:Thread-34-> Counter:8697
T

Thread Name:Thread-35-> Counter:8952
Thread Name:Thread-35-> Counter:8953
Thread Name:Thread-35-> Counter:8954
Thread Name:Thread-28-> Counter:8955
Thread Name:Thread-28-> Counter:8956
Thread Name:Thread-28-> Counter:8957
Thread Name:Thread-28-> Counter:8958
Thread Name:Thread-28-> Counter:8959
Thread Name:Thread-28-> Counter:8960
Thread Name:Thread-28-> Counter:8961
Thread Name:Thread-28-> Counter:8962
Thread Name:Thread-35-> Counter:8963
Thread Name:Thread-35-> Counter:8964
Thread Name:Thread-35-> Counter:8965
Thread Name:Thread-35-> Counter:8966
Thread Name:Thread-35-> Counter:8967
Thread Name:Thread-35-> Counter:8968
Thread Name:Thread-35-> Counter:8969
Thread Name:Thread-35-> Counter:8970
Thread Name:Thread-35-> Counter:8971
Thread Name:Thread-35-> Counter:8972
Thread Name:Thread-35-> Counter:8973
Thread Name:Thread-35-> Counter:8974
Thread Name:Thread-28-> Counter:8975
Thread Name:Thread-36-> Counter:8976
Thread Name:Thread-36-> Counter:8977
Thread Name:Thread-36-> Counter:8978
T

Thread Name:Thread-28-> Counter:9232
Thread Name:Thread-28-> Counter:9233
Thread Name:Thread-28-> Counter:9234
Thread Name:Thread-28-> Counter:9235
Thread Name:Thread-28-> Counter:9236
Thread Name:Thread-28-> Counter:9237
Thread Name:Thread-28-> Counter:9238
Thread Name:Thread-28-> Counter:9239
Thread Name:Thread-36-> Counter:9240
Thread Name:Thread-36-> Counter:9241
Thread Name:Thread-36-> Counter:9242
Thread Name:Thread-36-> Counter:9243
Thread Name:Thread-36-> Counter:9244
Thread Name:Thread-36-> Counter:9245
Thread Name:Thread-36-> Counter:9246
Thread Name:Thread-35-> Counter:9247
Thread Name:Thread-35-> Counter:9248
Thread Name:Thread-35-> Counter:9249
Thread Name:Thread-35-> Counter:9250
Thread Name:Thread-35-> Counter:9251
Thread Name:Thread-35-> Counter:9252
Thread Name:Thread-35-> Counter:9253
Thread Name:Thread-35-> Counter:9254
Thread Name:Thread-35-> Counter:9255
Thread Name:Thread-36-> Counter:9256
Thread Name:Thread-36-> Counter:9257
Thread Name:Thread-36-> Counter:9258
T

Thread Name:Thread-31-> Counter:9512
Thread Name:Thread-35-> Counter:9513
Thread Name:Thread-35-> Counter:9514
Thread Name:Thread-35-> Counter:9515
Thread Name:Thread-35-> Counter:9516
Thread Name:Thread-35-> Counter:9517
Thread Name:Thread-28-> Counter:9518
Thread Name:Thread-28-> Counter:9519
Thread Name:Thread-36-> Counter:9520
Thread Name:Thread-36-> Counter:9521
Thread Name:Thread-36-> Counter:9522
Thread Name:Thread-28-> Counter:9523
Thread Name:Thread-35-> Counter:9524
Thread Name:Thread-31-> Counter:9525
Thread Name:Thread-31-> Counter:9526
Thread Name:Thread-31-> Counter:9527
Thread Name:Thread-31-> Counter:9528
Thread Name:Thread-31-> Counter:9529
Thread Name:Thread-31-> Counter:9530
Thread Name:Thread-31-> Counter:9531
Thread Name:Thread-31-> Counter:9532
Thread Name:Thread-31-> Counter:9533
Thread Name:Thread-31-> Counter:9534
Thread Name:Thread-31-> Counter:9535
Thread Name:Thread-28-> Counter:9536
Thread Name:Thread-28-> Counter:9537
Thread Name:Thread-36-> Counter:9538
T

Thread Name:Thread-28-> Counter:9792
Thread Name:Thread-28-> Counter:9793
Thread Name:Thread-28-> Counter:9794
Thread Name:Thread-28-> Counter:9795
Thread Name:Thread-28-> Counter:9796
Thread Name:Thread-28-> Counter:9797
Thread Name:Thread-28-> Counter:9798
Thread Name:Thread-28-> Counter:9799
Thread Name:Thread-31-> Counter:9800
Thread Name:Thread-31-> Counter:9801
Thread Name:Thread-28-> Counter:9802
Thread Name:Thread-28-> Counter:9803
Thread Name:Thread-28-> Counter:9804
Thread Name:Thread-28-> Counter:9805
Thread Name:Thread-31-> Counter:9806
Thread Name:Thread-31-> Counter:9807
Thread Name:Thread-31-> Counter:9808
Thread Name:Thread-31-> Counter:9809
Thread Name:Thread-31-> Counter:9810
Thread Name:Thread-31-> Counter:9811
Thread Name:Thread-31-> Counter:9812
Thread Name:Thread-31-> Counter:9813
Thread Name:Thread-31-> Counter:9814
Thread Name:Thread-31-> Counter:9815
Thread Name:Thread-31-> Counter:9816
Thread Name:Thread-31-> Counter:9817
Thread Name:Thread-31-> Counter:9818
T

In [19]:
[t.join() for t in threads];

In [31]:
COUNTER

10000

In [32]:
assert COUNTER == (len(threads) * ITERATIONS), f"Invalid value for counter: {COUNTER}"

Perfect!

## Summary:

We've seen the importance of keeping our critical sections safe, to avoid race conditions. But there's no free lunch. To prevent race conditions we have to use synchronization mechanisms, and as we saw, that can carry other issues.

In the next section we'll explore one of the many things that can go wrong with manual synchronization: one of the the scariest words in computer science: **Deadlocks**.